In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import argparse
import time
import csv
import sys
import json
import random
import numpy as np
import pprint
import yaml
from torch import nn

import torch
import torch.multiprocessing as mp

import ray
from ray import tune

from matdeeplearn import models, process, training

config_path = 'config.yml'
#os.path.exists(config_path)
# os
os.path.abspath(os.getcwd())

assert os.path.exists(config_path), (
    "Config file not found in " + config_path
  )
with open(config_path, "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [2]:
#General imports
import csv
import os
import time
from datetime import datetime
import shutil
import copy
import numpy as np
from functools import partial
import platform
import random
import pandas as pd

##Torch imports
import torch.nn.functional as F
import torch
from torch_geometric.data import DataLoader, Dataset
from torch_geometric.nn import DataParallel
import torch_geometric.transforms as T
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.multiprocessing as mp

##Matdeeplearn imports
from matdeeplearn import models
import matdeeplearn.process as process
import matdeeplearn.training as training
from matdeeplearn.models.utils import model_summary

In [3]:
config['Job']['run_mode'] = 'Bootstrap_Pred'



In [4]:
config['Job'] = config['Job'].get(config['Job']['run_mode'])

In [5]:
config['Job']

{'job_name': 'my_bootstrap_job',
 'reprocess': 'False',
 'model': 'MEGNet_demo',
 'load_model': 'False',
 'save_model': 'True',
 'model_n': 'my_model.pth',
 'model_path': 'models_bootstrap',
 'write_output': 'True',
 'parallel': 'False',
 'boot_samples': 50,
 'seed': 0}

In [6]:
config["Models"] = config["Models"].get(config["Job"]['model'])
world_size = torch.cuda.device_count()
print(world_size)
config["Processing"]["data_path"] = "data/test_data/test_data_2"
rank = 'cuda'
print(world_size)
data_path = config["Processing"]["data_path"]
job_parameters= config["Job"]
training_parameters= config["Training"]
model_parameters= config["Models"]
processing_args= config['Processing']


1
1


In [101]:
config['Processing']["dictionary_source"]

'default'

In [153]:
dataset[0]

Data(edge_index=[2, 169], edge_weight=[169], y=-54.54595947265625, z=[13], u=[1, 3], structure_id=[1], x=[13, 114], edge_attr=[169, 50])

In [151]:
dataset = process.get_dataset(data_path, training_parameters["target_index"], 'False', config['Processing'])

In [7]:
##Get dataset
dataset = process.get_dataset(data_path, training_parameters["target_index"], False)

if rank not in ("cpu", "cuda"):
    dist.barrier()
n_samples = 50

##Set up loader
(
    t_loaders,
    val_loader,
    test_loader,
    train_sampler,
    train_sets,
    train_dataset,
    _,
    _,
) = training.loader_setup_bootstrap(
    training_parameters["train_ratio"],
    training_parameters["val_ratio"],
    training_parameters["test_ratio"],
    model_parameters["batch_size"],
    dataset,
    rank,
    job_parameters["seed"],
    world_size,
    num_samples = n_samples,
)

train length: 800 val length: 50 test length: 150 unused length: 0 seed : 0


/home/varivoda/.local/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
train_sets

In [115]:
dataset[0]

Data(edge_index=[2, 100], edge_weight=[100], y=-41.396141052246094, z=[10], u=[1, 3], structure_id=[1], x=[10, 114], edge_attr=[100, 50])

In [ ]:
print("Starting regular training")
print(
    "running for "
    + str(config["Models"]["epochs"])
    + " epochs"
    + " on "
    + str(config["Job"]["model"])
    + " model" 
    + " with "
    + str(config["Job"]["boot_samples"])
    + " bootstrapped "
    + " samples "
)
world_size = torch.cuda.device_count()
if world_size == 0:
    print("Running on CPU - this will be slow")
    training.train_regular_bootstrap(
        "cpu",
        world_size,
        config["Processing"]["data_path"],
        config["Job"],
        config["Training"],
        config["Models"],
        n_samples = config["Job"]["boot_samples"],
    )


elif world_size > 0:
    if config["Job"]["parallel"] == "True":
        print("Running on", world_size, "GPUs")
        mp.spawn(
            training.train_regular_bootstrap,
            args=(
                world_size,
                config["Processing"]["data_path"],
                config["Job"],
                config["Training"],
                config["Models"],
                config["Job"]["boot_samples"],
            ),
            nprocs=world_size,
            join=True,
        )
    if config["Job"]["parallel"] == "False":
        print("Running on one GPU")
        training.train_regular_bootstrap(
            "cuda",
            world_size,
            config["Processing"]["data_path"],
            config["Job"],
            config["Training"],
            config["Models"],
            config["Job"]["boot_samples"],
        )

Starting regular training
running for 250 epochs on MEGNet_demo model with 50 bootstrapped  samples 
Running on one GPU
train length: 15840 val length: 990 test length: 2970 unused length: 1 seed : 0


/home/varivoda/.local/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


--------------------------------------------------------------------------
               Layer.Parameter    Param Tensor Shape              Param #
--------------------------------------------------------------------------
         pre_lin_list.0.weight            [100, 114]                11400
           pre_lin_list.0.bias                 [100]                  100
       e_embed_list.0.0.weight             [100, 50]                 5000
         e_embed_list.0.0.bias                 [100]                  100
       e_embed_list.0.2.weight            [100, 100]                10000
         e_embed_list.0.2.bias                 [100]                  100
       e_embed_list.1.0.weight            [100, 100]                10000
         e_embed_list.1.0.bias                 [100]                  100
       e_embed_list.1.2.weight            [100, 100]                10000
         e_embed_list.1.2.bias                 [100]                  100
       e_embed_list.2.0.weight      

Epoch: 0005, Learning Rate: 0.000500, Training Error: 1.28747, Val Error: 1.51026, Time per epoch (s): 113.04389
Epoch: 0010, Learning Rate: 0.000500, Training Error: 0.77357, Val Error: 0.42736, Time per epoch (s): 113.01957
Epoch: 0015, Learning Rate: 0.000500, Training Error: 0.72529, Val Error: 1.18159, Time per epoch (s): 112.81088
Epoch: 0020, Learning Rate: 0.000500, Training Error: 0.51350, Val Error: 0.33562, Time per epoch (s): 113.23759
Epoch: 0025, Learning Rate: 0.000500, Training Error: 0.49338, Val Error: 1.06441, Time per epoch (s): 112.75641
Epoch: 0030, Learning Rate: 0.000500, Training Error: 0.56775, Val Error: 0.86047, Time per epoch (s): 113.38216
Epoch: 0035, Learning Rate: 0.000500, Training Error: 0.51867, Val Error: 0.38369, Time per epoch (s): 112.76163
Epoch: 0040, Learning Rate: 0.000500, Training Error: 0.46592, Val Error: 0.97911, Time per epoch (s): 112.74375
Epoch: 0045, Learning Rate: 0.000500, Training Error: 0.44139, Val Error: 1.02782, Time per epoc

In [84]:
dataset[0

RuntimeError: The 'data' object was created by an older version of PyG. If this error occurred while loading an already existing dataset, remove the 'processed/' directory in the dataset's root folder and try again.

In [138]:
tot_ids = train_dataset.indices
my_ids = []

for i in range(0,50):
    my_ids = my_ids + training_bootstrapped[i].indices
    
frame_dict = {'ids': np.unique(tot_ids).tolist(), 'count': np.repeat(0,len(np.unique(tot_ids).tolist()))}
freq = pd.DataFrame(frame_dict)




In [143]:
len(training_bootstrapped)

50

In [145]:
range(0, len(training_bootstrapped))

for i in range(0, len(training_bootstrapped)):
    if i > 45:
        print(i)

46
47
48
49


In [141]:
for item in my_ids: 
    if (item in freq['ids'].tolist()): 
        freq['count'][freq['ids'] == item] += 1
    else: 
        print('oopsie')

In [142]:
freq

,ids,count
0,1,62
1,2,49
2,3,55
3,4,38
4,5,46
...,...,...
795,994,48
796,996,49
797,997,44
798,998,43


In [116]:
dataset_size = len(dataset)
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
if (train_ratio + val_ratio + test_ratio) <= 1:
    train_length = int(dataset_size * train_ratio)
    val_length = int(dataset_size * val_ratio)
    test_length = int(dataset_size * test_ratio)
    unused_length = dataset_size - train_length - val_length - test_length
    (
        train_dataset,
        val_dataset,
        test_dataset,
        unused_dataset,
    ) = torch.utils.data.random_split(
        dataset,
        [train_length, val_length, test_length, unused_length],
        generator=torch.Generator().manual_seed(1),
    )


    total_ind = train_dataset.indices
    c_range = range(0, len(total_ind))
    

In [81]:
(
        train_dataset,
        val_dataset,
        test_dataset,
        unused_dataset,
    ) = torch.utils.data.random_split(
        dataset,
        [train_length, val_length, test_length, unused_length],
        generator=torch.Generator().manual_seed(1),
    )


In [117]:
total_ind

[845,
 303,
 66,
 436,
 239,
 593,
 573,
 999,
 919,
 268,
 228,
 955,
 892,
 60,
 457,
 485,
 310,
 904,
 402,
 998,
 170,
 95,
 702,
 169,
 739,
 976,
 886,
 155,
 333,
 672,
 928,
 585,
 439,
 572,
 911,
 788,
 403,
 592,
 454,
 719,
 565,
 987,
 651,
 828,
 199,
 555,
 478,
 790,
 425,
 53,
 64,
 354,
 584,
 622,
 977,
 503,
 583,
 47,
 607,
 384,
 926,
 869,
 363,
 43,
 517,
 393,
 519,
 762,
 840,
 784,
 359,
 86,
 884,
 927,
 516,
 648,
 249,
 352,
 269,
 164,
 391,
 89,
 90,
 189,
 34,
 40,
 803,
 320,
 962,
 138,
 910,
 311,
 197,
 837,
 931,
 477,
 823,
 366,
 662,
 626,
 753,
 482,
 65,
 347,
 368,
 726,
 968,
 481,
 791,
 431,
 62,
 649,
 297,
 783,
 453,
 897,
 542,
 213,
 582,
 455,
 992,
 566,
 527,
 250,
 873,
 829,
 243,
 276,
 848,
 415,
 469,
 864,
 30,
 853,
 556,
 768,
 148,
 159,
 195,
 862,
 891,
 458,
 492,
 251,
 988,
 909,
 240,
 903,
 484,
 694,
 705,
 984,
 940,
 380,
 536,
 87,
 118,
 832,
 429,
 274,
 815,
 322,
 813,
 184,
 421,
 244,
 152,
 3,
 153,
 223

In [121]:
my_indices = training.create_random_indices(total_ind, 50)
training_bootstrapped = []
seed = 0

for ind in my_indices:
    my_tr = copy.copy(train_dataset)
    my_tr.indices = ind
    training_bootstrapped.append(my_tr)


print(
    "train length:",
    train_length,
    "val length:",
    val_length,
    "test length:",
    test_length,
    "unused length:",
    unused_length,
    "seed :",
    seed,
)

train length: 800 val length: 100 test length: 100 unused length: 0 seed : 0


In [136]:
training_bootstrapped = []


for ind in my_indices:
    my_tr = copy.copy(train_dataset)
    my_tr.indices = ind
    training_bootstrapped.append(my_tr)

In [104]:
my_indices = training.create_random_indices(train_dataset.indices, 50)

In [132]:
my_tr = copy.copy(train_dataset)
my_tr.indices = my_indices[0]

In [133]:
my_tr[0]

Data(edge_index=[2, 100], edge_weight=[100], y=-38.59280014038086, z=[10], u=[1, 3], structure_id=[1], x=[10, 114], edge_attr=[100, 50])

In [134]:
my_tr2 = copy.copy(train_dataset)
my_tr2.indices = my_indices[1]

In [135]:
my_tr2[0]

Data(edge_index=[2, 100], edge_weight=[100], y=-41.23038101196289, z=[10], u=[1, 3], structure_id=[1], x=[10, 114], edge_attr=[100, 50])